This is an example script for the PhotoReceptor model. Please run each cell in sequence. 

Warnings about "replacing module" when exectuing the first cell can be ignored.

If you want to change how many iterations are run, you can do so in the second cell.
- nproposals: number of proposals evaluated in parallel in the Generalized Metropolis Hastings algorithm
- niterations: number of total iterations

The values given below (1000 proposals for 100 iterations) will take about 2.5 minutes to run on an 8-core machine.

Remember that you can remove output from the cells by selecting "Cell/All Output/Clear". And if the Kernel hangs for some reason, then you can choose "Kernel/Restart" from the menu.

In [ ]:
###You will need to run this cell only once, unless some workers crash and shut down
NPROCS = 16 #make this the number of processors on your machine. For JuliaBox, it is 16
if nprocs() < NPROCS
    addprocs(NPROCS-nprocs())
end
println("Number of parallel processes: ",nprocs())

###Load the GMHExamples package on all processes
@everywhere include("../../GMH-Examples.jl")

###Load the PyPlot package
using PyPlot
println("PyPlot package loaded successfully")

In [ ]:
println("Number of parallel processes running: ",nprocs())

using GeneralizedMetropolisHastings
using GMHExamples

println("================================")
println("Initialize Simulation Parameters")
println("================================")

#Standard M-H for nproposals == 1
#Generalized M-H for nproposals > 1
nproposals = 1000

#MCMC iteration specifications
nburnin = 0
niterations = 100
ntunerperiod = 5

###Values of the model
numvilli1 = 30000

#specify the values that determine the priors on the parameters
latencylocation = (2.0,3.5) #uniform distribution with (low,high) values
latencyscale = (0.2,0.7) #uniform distribution with (low,high) values
refractorylocation = (4.0,6.0) #uniform distribution with (low,high) values
refractoryscale = (1.5,2.5) #uniform distribution with (low,high) values

photons1 = photonsequence("../data/naturallight.jld")
current1 = lightinducedcurrent("../data/naturallight.jld")

modelpolicy1 = policy(:photoreceptor) #4-parameter model with stochastic lognormal latency and refractory parameters
params1 = parameters(:photoreceptor,modelpolicy1,latencylocation,latencyscale,refractorylocation,refractoryscale)

####Variance for the noise model, estimated from previous runs
variance1 = [3600.0]

println("==========================================")
println("Simulation parameters defined successfully")
println("==========================================")

In [ ]:
###Create a PhotoReceptor model
model1 = model(:photoreceptor,params1,photons1,current1,variance1,numvilli1,modelpolicy1)

###Show the model
println("==========================")
println("Model defined successfully")
println("==========================")
show(model1)

In [ ]:
###Plot the measurement data (simmulated data + noise)
figure("PhotoReceptor1") ; clf()
plot(dataindex(model1),measurements(model1);label="measured",linewidth=2)
xlabel("Time")
ylabel("Current (nA)")
xlim(dataindex(model1)[1],dataindex(model1)[end])
title("Light-Induced Current")
grid("on")
legend(loc="upper right",fancybox="true")

In [ ]:
###Create a Metropolis sampler with a Normal proposal density
propcov = [0.1,0.01,0.1,0.01]
sampler1 = sampler(:mh,:normal,1.0,propcov)
println("============================")
println("Sampler defined successfully")
println("============================")
show(sampler1)
println(" initialscalefactor: $(sampler1.initialscalefactor)")
println(" covariance: $(sampler1.covariance)")
println()

###Create a tuner, either one that scales the stepsize of the proposal density or one that monitors acceptance rates
#tuner1 = tuner(:scale,ntunerperiod,0.5,:erf)
tuner1 = tuner(:monitor,ntunerperiod)
println("==========================")
println("Tuner defined successfully")
println("==========================")
show(tuner1)

###Create a Generalized Metropolis-Hastings runner (which will default to Standard MH when nproposals=1)
runnerpolicy1 = policy(:gmh,nproposals;initialize=:prior)
runner1 = runner(:gmh,niterations,nproposals,runnerpolicy1;numburnin = nburnin)
println("===========================")
println("Runner defined successfully")
println("===========================")
show(runner1)

In [ ]:
###Run the MCMC (can take quite a bit of time)
println("=======================")
println("Run the MCMC simulation")
println("=======================")
chain1 = run!(runner1,model1,sampler1,tuner1)
println("=========================")
println("Completed MCMC simulation")
println("=========================")

In [ ]:
###Show the result of the simulations
show(chain1)
nparas = numparas(model1)
meanparamvals = mean(samples(chain1),2)
stdparamvals = std(samples(chain1),2)

println("Results of the MCMC simulation:")
for i=1:nparas
    println("mean $(parameters(model1)[i].key): $(meanparamvals[i])")
    println("std $(parameters(model1)[i].key): $(stdparamvals[i])")
end

In [ ]:
println("================")
println("Plotting results")
println("================")

###Plot the average model results in the data window
subplot(211)
modeldata = evaluate!(model1,vec(meanparamvals))
plot(dataindex(model1),measurements(model1);label="measured",linewidth=4,color="yellow")
plot(dataindex(model1),modeldata;label="model fit",linewidth=1,color="black")
xlim(dataindex(model1)[1],dataindex(model1)[end])
legend(loc="upper right",fancybox="true")

###Plot the logposterior values across samples
subplot(212)
plot(1:numsamples(chain1),logposterior(chain1,model1))
title("Log-Posterior values across samples")
xlabel("Samples")
ylabel("Log-Posterior")



In [ ]:
###Plot the histograms of parameter values
for i=1:nparas
    subplot(220 + i)
    bins = meanparamvals[i]-0.2:0.01:meanparamvals[i]+0.2
    h = PyPlot.plt[:hist](sub(samples(chain1),i,:)',bins)
    grid("on")
    title("$(parameters(model1)[i].key)")
    if i > 2
        xlabel("Values")
    end
    if i == 1 || i == 3
        ylabel("Samples")
    end
    xlim(meanparamvals[i]-0.2,meanparamvals[i]+0.2)
end

In [ ]:
###Only run this box if you want to shut down all worker processes
println("Pre processes running: ",procs())
for i in workers()
    if i != 1
        rmprocs(i)
    end
end
println("Post processes running: ",procs())